# Project enda : Example D


Install all the required packages in your python virtualenv:
```bash
pip install numexpr bottleneck pandas enda jupyter h2o scikit-learn statsmodels matplotlib joblib
# more packages used for feature engineering below
pip install jours-feries-france vacances-scolaires-france Unidecode 
```

In this example we will set up a simple dayahead energy production prediction.  

We set ourselves in a setup as if we were **exactly on 2021-01-01**. We want to predict the production of several power plants for the next day on 30 min time-step intervals. In this example, we have exactly one year of data that spans the whole year 2020. This data is split in several files that are likely to be the ones obtained from a typical ETL processing. We always separate the data between the type of power plant (solar, wind, run of river), because they have very different behaviour, as we'll see. We thus consider:

- a list of stations with their associated installed_capacity (`wind_stations.csv`, `historic_solar.csv`, `historic_river.csv`). These files summarize contracts we may have with the aforementioned power stations. 

- the historical power generation data for the power stations along the year 2020. Note this has to be obtained by yourself according to your needs (as an example, Enercoop data are regularly published from the French TSO)

- the historical meteo data for the power stations along the year 2020. This also has to be obtained on your side (as an example, we regularly gather meteo information from GFS using the zipcode of the power plants in our portfolio).

- a list of events (planned shutdowns, outages) that may have dirupted the usual installed capacity of the power stations. 


We will : 
- set up the relevant training datasets and do some feature engineering ;
- set up several models of training ;
- predict the dayahead energy production per power plant, and its aggregated counterpart. 

In [ ]:
import pandas as pd
import os
import enda
import datetime
import numpy as np
from pandas.api.types import is_string_dtype
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 30
enda.__file__

In [ ]:
DIR = '/Users/clement.jeannesson/Jobs/enda/'
DIR_TEST = '/Users/clement.jeannesson/Jobs/enda/tests/example_d/input_example/'

## Portfolio

In [ ]:
# The details of power stations can be interpreted as enda contracts. 
stations_wind =  enda.Contracts.read_contracts_from_file(os.path.join(DIR_TEST, "wind_stations.csv"))
stations_solar = enda.Contracts.read_contracts_from_file(os.path.join(DIR_TEST, "solar_stations.csv"))
stations_river = enda.Contracts.read_contracts_from_file(os.path.join(DIR_TEST, "river_stations.csv"))
stations=[stations_wind, stations_solar, stations_river]

In [ ]:
_ = [display(station) for station in stations]

For this example, we have chosen to consider four power stations of each type. 

Exactly as contracts data, we have a date start and a date of end, and some characteristics that stay valid over that time lap. The end date is properly set in most cases. This differs from consumption contracts (cf. Example A), for which no end date are provided.  

We want to get a daily data for these stations. Here, a change of installed capacity is spotted for ```eo_4```. This has to be taken care of.

In [ ]:
# all enda functions in this notebook example are defined through wrappers

def get_stations_daily(df):
    return enda.PowerStations.get_stations_daily(
               df, 
               station_col='station',
               date_start_col="date_start",
               date_end_exclusive_col="date_end_exclusive"
           )

stations_daily = [get_stations_daily(station) for station in stations]
display(stations_daily[0])

In [ ]:
# get the portfolio between dates

def get_stations_between_dates(df):
    return enda.PowerStations.get_stations_between_dates(
               df, 
               start_datetime = pd.to_datetime('2020-01-01'),
               end_datetime_exclusive = pd.to_datetime('2021-01-01')
           )

stations_daily = [get_stations_between_dates(station) for station in stations_daily]
display(stations_daily[0])

In [ ]:
# let's check it's ok for wind station 
stations_wind_daily = stations_daily[0]
dates_test = [datetime.datetime(2020, 2, 17) + datetime.timedelta(days=x) for x in range(4)]
stations_wind_daily[(stations_wind_daily.index.get_level_values("station") == "eo_4") &
                    (stations_wind_daily.index.get_level_values("date").isin(dates_test))
                   ]

In [ ]:
# interpolate data to a sub-daily scale using enda.

def interpolate_stations_to_subdaily_data(df):
    return enda.TimeSeries.interpolate_daily_to_sub_daily_data(
               df,
               freq='30min', 
               tz='Europe/Paris', 
               index_name='time'
           )

stations_finergrid = [interpolate_stations_to_subdaily_data(station) for station in stations_daily]
display(stations_finergrid[0])

At this point we have the portfolio information per day. It is useful to take into account the outages and shutdowns.

### Take into account outages

In [ ]:
# read outages 
filepath =  os.path.join(DIR_TEST, "events.csv")

outages = enda.PowerStations.read_outages_from_file(
    filepath, 
    station_col='station',
    time_start_col="time_start", 
    time_end_exclusive_col="time_end", 
    pct_outages_col="impact_production_pct_kw",
    tzinfo="Europe/Paris"
)


In [ ]:
display(outages)

In [ ]:
# wrapper to integrate the outages to the stations portfolio.
def integrate_outages_to_stations(df):
    return enda.PowerStations.integrate_outages(
        stations=df,   
        outages=outages, 
        station_col="station",
        time_start_col="time_start",
        time_end_exclusive_col="time_end", 
        installed_capacity_col="installed_capacity_kw",
        pct_outages_col="impact_production_pct_kw"
    )

stations_historic = [integrate_outages_to_stations(station) for station in stations_finergrid]
display(stations_historic[0])

In [ ]:
# check the outages have been corectly taken into account 
stations_wind_daily = stations_historic[0]
stations_wind_daily.loc[(stations_wind_daily.index.get_level_values("station") == "eo_2")
                    & (stations_wind_daily.index.get_level_values("time") >= pd.to_datetime('2020-02-23 22:00:00+01:00'))
                    & (stations_wind_daily.index.get_level_values("time") < pd.to_datetime('2020-02-24 02:00:00+01:00'))
                   ]

# Meteo and production

Here, the meteo and production data have been retrived over the year 2020.  
We need to set them on the frequency of interest, which is a 30 minutes time lap. 

In [ ]:
# retrieve meteo information. We only have it for solar and wind stations
meteo_wind = pd.read_csv(os.path.join(DIR_TEST, "historic_meteo_wind.csv"))
meteo_solar = pd.read_csv(os.path.join(DIR_TEST, "historic_meteo_solar.csv"))

for df in [meteo_wind, meteo_solar]:
    df['time'] = pd.to_datetime(df['time'])
    df['time'] = enda.TimeSeries.align_timezone(df['time'], tzinfo = 'Europe/Paris')
    df.set_index(["station", "time"], inplace=True)
    
meteo = [meteo_wind, meteo_solar]
_ = [display(_) for _ in meteo]

Please note meteo data cannot be separated from the power plant, because of the meteo is related specific location of each plant. 

In [ ]:
# interpolate the meteo information to a 30 minutes scale
def apply_freq_to_sub_freq(df):
    return enda.TimeSeries.interpolate_freq_to_sub_freq_data(
               df,
               freq='30min', 
               tz='Europe/Paris',
               index_name='time',
               method="linear"
           )

meteo_wind, meteo_solar = (apply_freq_to_sub_freq(m) for m in meteo[0:2])
meteo_historic = [meteo_wind, meteo_solar]

In [ ]:
meteo_wind

Get production information. This information is avaliable from the TSO. It is provided on a fine 10-minutes timestep, and we need to average it over the half-hour scale.

In [ ]:
# retrieve production information.
production_wind = pd.read_csv(os.path.join(DIR_TEST, "historic_production_wind.csv"))
production_solar = pd.read_csv(os.path.join(DIR_TEST, "historic_production_solar.csv"))
production_river = pd.read_csv(os.path.join(DIR_TEST, "historic_production_river.csv"))

for df in [production_wind, production_solar, production_river]:
    df['time'] = pd.to_datetime(df['time'])
    df['time'] = enda.TimeSeries.align_timezone(df['time'], tzinfo = 'Europe/Paris')
    df.set_index(["station", "time"], inplace=True)

production = [production_wind, production_solar, production_river]

In [ ]:
# let us display production for run of river stations
display(production_river)

In [ ]:
# Let us average the production over a 30 minutes time scale.
def apply_average_to_upper_freq(df):
    return enda.TimeSeries.average_to_upper_freq(
               df,
               freq='30min', 
               tz='Europe/Paris',
               index_name='time'
           )

production_wind, production_solar, production_river = (apply_average_to_upper_freq(prod) for prod in production)
production_historic = [production_wind, production_solar, production_river]

In [ ]:
display(production_river)

We gathered information about the power stations in our example portfolio over the year 2020, as well as production data, and meteo information (for solar and wind only). We managed to set them on a 30-minutes scale. We need to merge these data together, to produce training sets that will be useful for our prediction. 

In [ ]:
# wrapper to merge the portfolio, production, and meteo dataframes for each type 
# of power plant.
def merge_stations_and_features(df1, df2):
    df = pd.merge(df1, df2, how='left', left_index=True, right_index=True)
    return df.dropna()
    
full_historic = [merge_stations_and_features(station, production)
                 for station, production in zip(stations_historic, production_historic)]

full_historic[0:2] = [merge_stations_and_features(station, meteo)
                      for station, meteo in zip(full_historic[0:2], meteo_historic)]

In [ ]:
# Let us display all the dataframes
_ = [display(historic) for historic in full_historic]

## Featurize

Let's try to add some feature for the solar information, namely the cos and sin of dates, as what has been done in the Example 2.

In [ ]:
# using enda.feature_engineering.calendar for solar
full_historic[1] = enda.DatetimeFeature.split_datetime(
        full_historic[1], split_list = ['minuteofday', 'dayofyear']
    )

full_historic[1] = enda.DatetimeFeature.encode_cyclic_datetime_index(
        full_historic[1], split_list = ['minuteofday', 'dayofyear']
    )

display(full_historic[1])

## Compute the load_factor

The *load factor* is the target of the algorithm. 
Let's compute it from the `installed_capacity` and the `power_kw` fields

In [ ]:
# wrapper to compute the load factor. 
# We drop the power_kw information during that step.
def compute_load_factor(df):
    return enda.PowerStations.compute_load_factor(
               df, 
               installed_capacity_kw='installed_capacity_kw', 
               power_kw='power_kw',
               drop_power_kw=True
           )           

train_data = (compute_load_factor(historic) for historic in full_historic)
train_wind, train_solar, train_river = train_data


In [ ]:
display(train_wind, train_solar, train_river)

We have here the training datasets we'll use to make our prediction, which have been built using the enda utilities function, and some historical information gathered from the TSO, diverse meteo information suppliers, and contracts data. 

# Make a prediction

Let's use the enda algorithms to make a simple power prediction. 

First of all, let us retrieve the forecast dataframes used for our prediction. They contain portfolio and meteo information for the 02/02/2021. They actually have been processed following a similar pipeline than the the one presented in this notebook for the training dataframes.

In [ ]:
# retreive forecast data
forecast_wind = pd.read_csv(os.path.join(DIR_TEST,  "forecast_wind.csv"))
forecast_solar = pd.read_csv(os.path.join(DIR_TEST, "forecast_solar.csv"))
forecast_river = pd.read_csv(os.path.join(DIR_TEST, "forecast_river.csv"))

for df in [forecast_wind, forecast_solar, forecast_river]:
    df['time'] = pd.to_datetime(df['time'])
    df['time'] = enda.TimeSeries.align_timezone(df['time'], tzinfo = 'Europe/Paris')
    df.set_index(["station", "time"], inplace=True)

forecast = [forecast_wind, forecast_solar, forecast_river]
display(forecast_wind, forecast_solar, forecast_river)

We need to import the ML backends from enda, as well as the anda wrapper meant to handle calculations specific to the power prediction, ie. objects of the class PowerPredictor. This class enables to use EndaEstimators for all power plants considered as individual plants, or instances of a theoretical standard one. That latter approach is the standard power plant method, in which all observations are merged together to form a training dataset, and individual prperties of each plant are considered as additional features of the algorithm (this is notably the case of the `installed_capacity`column in the present example.

Here, we will use linear predictors coupled with a standard power plant approach for the solar and wind stations. For the hydraulic plants, the chosen methodology will be slightly different, as we use in practice a much more naive technique, which is simply a recopy of the last observation for each power plant. This means using a non standard power plant approach coupled with the so-called `Recopy()` enda estimator.

In [ ]:
# import ML backends
from enda.ml_backends.sklearn_estimator import EndaSklearnEstimator
from sklearn.linear_model import LinearRegression
from enda.estimators import EndaEstimatorRecopy

# import power predictors
from enda.power_predictor import PowerPredictor

### Run of river prediction

In [ ]:
# build a PowerPredictor obejct
river_predictor = PowerPredictor(standard_plant=False)

# use PowerPredictor to train the estimator from the run of river data, 
# and from a naive recopy estimator
river_predictor.train(train_river, estimator=EndaEstimatorRecopy(period='1D'), target_col="load_factor")

In [ ]:
# To see the guts of what's happening inside: as standard plant is False, a single estimator is created 
# for each power plant.
# It is trained individually on the available data; here, we need to naively recopy the data. 
# The prod_estimators field of the instance of PowerPredictor is a dictionary with the station ID, 
# and the estimator that we can train. Here we can access the fields training_data specific to 
# EndaEstimatorRecopy()
_ = [display(x[0], pd.DataFrame(x[1].training_data.T)) for x in river_predictor.prod_estimators.items()]

In [ ]:
# Once it has been trained, we can predict the power for each power plant individually, calling predict()
# from PowerPredictor()
pred_river = river_predictor.predict(forecast_river, target_col="load_factor")
pred_river

### Wind prediction

For the wind prediction, we will use a Sklearn Linear predictor. 

In [ ]:
# build a PowerPredictor object
wind_predictor = PowerPredictor(standard_plant=True)

# use a SkLearn Linear Regression estimator
lin_reg = EndaSklearnEstimator(LinearRegression())

# train the estimator
wind_predictor.train(train_wind, estimator=lin_reg, target_col="load_factor")

# predict
pred_wind = wind_predictor.predict(forecast_wind, target_col="load_factor")
pred_wind

### Solar prediction

for the solar prediction, we can use a linear regression model, or even a naive recopy model, still using a standard power plant approach. Note that this is only made for the purpose of this example, because such a naive estimator is the simplest one we can think of. 

In [ ]:
# build a PowerPredictor object
solar_predictor = PowerPredictor(standard_plant=True)

# train the estimator
solar_predictor.train(train_solar, estimator=EndaEstimatorRecopy(period='1D'), target_col="load_factor")

# predict
pred_solar = solar_predictor.predict(forecast_solar, target_col="load_factor")
pred_solar

## Getting back to power prediction

To get back to power prediction, we simply need to use the installed capacity field and multiply it by the load factor to find again the power (kw)

In [ ]:
# we start by merging again the installed_capacity (kw) field
prediction_river = merge_stations_and_features(forecast_river.loc[:, ["installed_capacity_kw"]], pred_river)
prediction_wind = merge_stations_and_features(forecast_wind.loc[:, ["installed_capacity_kw"]], pred_wind)
prediction_solar = merge_stations_and_features(forecast_solar.loc[:, ["installed_capacity_kw"]], pred_solar)

prediction_load_factor = [prediction_river, prediction_solar, prediction_wind]

prediction_wind

In [ ]:
# wrapper around compute_power_kw_from_load_factor
# We drop the load_factor information during that step.
def compute_power_kw_wrapper(df):
    return enda.PowerStations.compute_power_kw_load_factor(
               df, 
               installed_capacity_kw='installed_capacity_kw', 
               load_factor='load_factor',
               drop_load_factor=True
           )           

predicted_data = (compute_power_kw_wrapper(pred) for pred in prediction_load_factor)
predicted_wind, predicted_solar, predicted_river = predicted_data
predicted_river

## Conclusion 

We have been able to build a simple prediction using (or not) a standard power plant approach for a portfolio of plants of different types. It is possible to go further, notably performing a backtesting to explore the performance of the algorithms. 